In [1]:
# Лабораторная по рекомендательным системам

In [2]:
import json

import pandas as pd
import numpy as np

from scipy.sparse import csr_matrix
import matplotlib.pyplot as plt

%matplotlib inline

## 1. Подготовка данных

In [3]:
JSON_DATA_PATH = "Video_Games_5.json"
N = 10

def iter_json_data(path):
    with open(path) as f:
        for line in f:
            data = json.loads(line)
            yield data
            
def get_data_frame():
    uid_to_id = {}
    iid_to_id = {}
    
    cols = ["uid", "iid", "review", "rating", "dt"]
    rows = []
    for d in iter_json_data(JSON_DATA_PATH):
        uid = uid_to_id.setdefault(d["reviewerID"], len(uid_to_id))
        iid = iid_to_id.setdefault(d["asin"], len(iid_to_id))
        review = d["reviewText"]
        rating = float(d["overall"])
        dt = int(d["unixReviewTime"])
        rows.append((uid, iid, review, rating, dt))
        
    return pd.DataFrame(rows, columns=cols)

def split_df_by_dt(df, p=0.8):
    """Функция разбивает df на тестовую и тренировочную выборки по времени 
    публикации отзывов (значение времени в поле dt)
    
    :param p: персентиль значений dt, которые образуют тренировочную выборку. Например p=0.8 означает, что в 
    тренировочной части будут отзывы, соответствующие первым 80% временного интервала 
    :return: два pd.DataFrame объекта
    """
    border_dt = df.dt.quantile(p)
    print("Min=%s, border=%s, max=%s" % (df.dt.min(), border_dt, df.dt.max()))
    training_df, test_df  = df[df.dt <= border_dt], df[df.dt > border_dt]
    print("Размер до очистки:", training_df.shape, test_df.shape)
    # удаляем из тестовых данных строки, соответствующие пользователям или объектам, 
    # которых нет в тренировочных данных 
    # (пользователи - избегаем проблем для персональных систем, объекты - для всех)
    test_df = test_df[test_df.uid.isin(training_df.uid) & test_df.iid.isin(training_df.iid)]
    print("Размер после очистки:", training_df.shape, test_df.shape)
    return training_df, test_df

Загружаем данные.

In [4]:
df = get_data_frame()

# uid - идентификатор пользователя
# iid - идентификатор объекта
df.head()

,uid,iid,review,rating,dt
0,0,0,Installing the game was a struggle (because of...,1.0,1341792000
1,1,0,If you like rally cars get this game you will ...,4.0,1372550400
2,2,0,1st shipment received a book instead of the ga...,1.0,1403913600
3,3,0,"I got this version instead of the PS3 version,...",3.0,1315958400
4,4,0,I had Dirt 2 on Xbox 360 and it was an okay ga...,4.0,1308009600


Разделеяем данные на тренировочные и тестовые.

In [5]:
training_df, test_df = split_df_by_dt(df)
del df

Min=939859200, border=1377129600.0, max=1405987200
Размер до очистки: (185427, 5) (46353, 5)
Размер после очистки: (185427, 5) (19174, 5)


## 2. Метрика качества и базовый класс модели

Для оценки качества системы воспользуемся метрикой `hit-ratio (HR)`. 

$$
HR = \frac{1}{|U_T|}\sum_{u \in U_T} \mathrm{I}(Rel_u \cap Rec_u)
$$

* $U_T$ - множество пользователей из тестовой выборки
* $Rec_u$ - множество объектов, рекомендованных пользователю $u$ 
* $Rel_u$ - множество объектов, оцененных пользователем $u$ в тестовой выборке
* $\mathrm{I}(Rel_u \cap Rec_u)$ - бинарная функция-индикатор. Функция возвращает 1 если $Rel_u \cap Rec_u \ne \emptyset$, иначе 0

$HR=1$ если для каждого пользователя мы рекомендовали хотя бы один релевантный объект. Так как обычно пользователи просматривают только первые $N$ рекомендаций, мы будем считать метрику $HR@N$, где $N=10$ (т.е. множество $Rec_u$ будет содержать только 10 объектов). 

In [6]:
def hit_ratio(recs_dict, test_dict):
    """Функция считает метрику hit-ration для двух словарей
    :recs_dict: словарь рекомендаций типа {uid: {iid: score, ...}, ...}
    :test_dict: тестовый словарь типа {uid: {iid: score, ...}, ...}
    """
    hits = 0
    for uid in test_dict:
        if set(test_dict[uid].keys()).intersection(recs_dict.get(uid, {})):
            hits += 1
    return hits / len(test_dict)

def get_test_dict(test_df):
    """Функция, конвертирующая тестовый df в словарь
    """
    test_dict = {}
    for t in test_df.itertuples():
        test_dict.setdefault(t.uid, {})
        test_dict[t.uid][t.iid] = t.rating
    return test_dict

Преобразуем тестовый набор в словарь

In [7]:
test_dict = get_test_dict(test_df)

Создадим базовые классы для рекомендателей, которые можно использовать при построении собственных моделей.

In [8]:
class BasicRecommender(object):
    def __init__(self):
        pass
    
    def get_recs(self, uid, top):
        """Строит рекомендации для пользователя uid
        :return: словарь типа {iid: score, ...}
        """
        return {}
    
    def get_batch_recs(self, uids, top):
        """Строит рекомендации для нескольких пользователей uids
        :return: словарь типа {uid: {iid: score, ...}, ...}
        """
        return {uid: self.get_recs(uid, top) for uid in uids}
    
class NonPersRecommender(BasicRecommender):
    def __init__(self, df):
        super(NonPersRecommender, self).__init__()
        self.recs = self._prepare_recs(df)
        
    def _prepare_recs(self, df):
        return pd.Series([])
    
    def get_recs(self, uid, top):
        return self.recs[:top].to_dict()
    
    def get_batch_recs(self, uids, top):
        non_pers_recs = self.get_recs(None, top)
        return {uid: non_pers_recs for uid in uids}

## 3. Non-personalized RS

Неперсонализированная рекомендательная система - рекомендации для одного пользователя строятся на основе отзывов, оставленных всеми пользователями.


### 3.1. RS, рекомендующая наиболее популярный контент

In [9]:
class MostReviewedRS(NonPersRecommender):
    def _prepare_recs(self, df):
        # считаем количество отзывов для каждого объекта (pandas сортирует их по убыванию)
        return df.iid.value_counts()  

In [10]:
model = MostReviewedRS(training_df)
recs = model.get_batch_recs(test_df['uid'], N)

hit_ratio(recs, test_dict)

0.03961848862802641

### 3.2. RS, рекомендующая наиболее контент c наибольшим средним рейтингом

In [11]:
class HighestRatingRS(NonPersRecommender):
    def _prepare_recs(self, df):
        return df.groupby('iid').rating.mean().sort_values(ascending=False)

In [12]:
model = HighestRatingRS(training_df)
recs = model.get_batch_recs(test_df['uid'], N)

hit_ratio(recs, test_dict)

0.001467351430667645

### 3.3. RS, рекомендующая популярный контент на основе последних обзоров

In [13]:
class TrendyRS(NonPersRecommender):
    def __init__(self, df, recsN):
        super(NonPersRecommender, self).__init__()
        self.recs = self._prepare_recs(df, recsN)
        
    def _prepare_recs(self, df, recsN):
        return df.sort_values('dt')[-recsN:].iid.value_counts()

Рекомендация на оcнове **последних 10000** обзоров

In [14]:
model = TrendyRS(training_df, 10000)
recs = model.get_batch_recs(test_df['uid'], N)

hit_ratio(recs, test_dict)

0.09655172413793103

In [15]:
class TrendyRS_days(NonPersRecommender):
    def __init__(self, df, daysN):
        super(NonPersRecommender, self).__init__()
        self.recs = self._prepare_recs(df, daysN)
        
    def _prepare_recs(self, df, daysN):
        return df.sort_values('dt')[df['dt'] >= int(df.iloc[-1:].dt - daysN*86400)].iid.value_counts()

Рекомендация на оcнове обзоров за **последние 100 дней** 

In [16]:
model = TrendyRS_days(training_df, 100)
recs = model.get_batch_recs(test_df['uid'], N)

hit_ratio(recs, test_dict)

D:\Skaarj\Documents\Anaconda3\lib\site-packages\ipykernel\__main__.py:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


0.10359501100513573

Лучшая оценка для неперсонализированной RS - **0.10359501100513573**, модель на основе самых популярных отзывов за последние 100 дней.

## 4. Content-based RS

Рекомендации строятся путем поиска объектов, чьи вектора похожи на вектор предпочтений пользователя.

### 4.1. Векторизация описания игр

Нормализуем все описания: исключим стоп-слова, приведем к нижнему регистру, выделим леммы и т.п.

In [156]:
import re
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer

stopword_list = nltk.corpus.stopwords.words('english')
stopword_list = stopword_list + ['game', 'games']
x = re.compile('[%s]' % re.escape(string.punctuation + string.digits))
pst = PorterStemmer()
wlem = WordNetLemmatizer()

tokenized_docs = [word_tokenize(doc) for doc in training_df.loc[:,'review']]
normalized_docs = []

for review in tokenized_docs:
    #new_review = []
    new_review = ''
    for token in review:
        token = token.lower()
        
        new_token = x.sub(u'', token)
        if (not new_token == u'') and (new_token not in stopword_list) and (len(new_token) > 2) and (len(new_token) < 21):
            #new_review.append(wlem.lemmatize(pst.stem(new_token)))
            new_review += wlem.lemmatize(pst.stem(new_token)) + ' '
            
    normalized_docs.append(new_review)

In [162]:
training_df["normalized"] = normalized_docs
training_df.loc[:1,:]

,uid,iid,review,rating,dt,normalized
0,0,0,Installing the game was a struggle (because of...,1.0,1341792000,instal struggl window live bug championship ra...
1,1,0,If you like rally cars get this game you will ...,4.0,1372550400,like ralli car get funit orient european marke...


Построим TF-IDF на нормализованных данных

In [164]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(stop_words='english')
tfidf.fit(training_df.loc[:,'normalized'])

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words='english', strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

Составим коллекцию векторов всех игр из отзывов, выбросив игры у которых меньше 5 отзывов

In [209]:
from scipy.sparse import csr_matrix
from scipy.sparse import vstack

iids = training_df.groupby('iid').uid.count()
iids = iids[iids > 5].reset_index()
iids.columns=['iid', 'num']

X_g = []

for iid in iids.iid:
    tmp = tfidf.transform(training_df.loc[training_df.iid==iid].normalized)
    tmp = csr_matrix(tmp.sum(axis=0))
    X_g.append(tmp)
    
X_g = vstack(X_g, 'csr')

### 4.2. Создание профилей пользователей

In [234]:
uids = test_df.uid.unique()

X_u = []

for uid in uids:
   
    u_mean_rating = training_df.loc[training_df.uid == uid].rating.mean()
    u_iids = training_df.loc[training_df.uid == uid].iid.unique()
    
    u_profile = []

    for iid in u_iids:
        rating = training_df.loc[(training_df.uid == uid) & (training_df.iid == iid)].rating.values[0] - u_mean_rating
        
        tmp = tfidf.transform(training_df.loc[training_df.iid==iid].normalized) * rating
        tmp = csr_matrix(tmp.sum(axis=0))
        u_profile.append(tmp)

    u_profile = vstack(u_profile, 'csr')
    
    X_u.append(csr_matrix(u_profile.sum(axis=0)))

X_u = vstack(X_u, 'csr')

### 4.3. Построение рекомендаций

In [309]:
class ContRS_cosine(BasicRecommender):
    def __init__(self, df_i, df_u, x_i, x_u):
        from sklearn.metrics.pairwise import cosine_similarity
        self.items = df_i
        self.users = df_u
        self.X_items = x_i
        self.X_users = x_u
        super(ContRS_cosine, self).__init__()
    
    def get_recs(self, uid, top):
        x_uid = np.where(uids == uid)
        
        sims = cosine_similarity(self.X_users[x_uid], self.X_items)
        best_iid = np.argsort(sims[0])[-(top+1):-1]
        true_x_iid = [iids.ix[x_iid,'iid'] for x_iid in best_iid]
               
        return dict(zip(true_x_iid, sims[0, best_iid]))
    
    def get_batch_recs(self, top):
        return {uid: self.get_recs(uid, top) for uid in self.users[0:20]}

In [ ]:
recs = {}

for x_u in range(0,X_u.shape[0]):
    sims = cosine_similarity(X_u[x_u], X_g)
    
    best_iid = np.argsort(sims[0])[-(10+1):-1]
    true_x_iid = [iids.ix[x_iid,'iid'] for x_iid in best_iid]
    
    recs[np.str(uids[x_u])] = dict(zip(true_x_iid, sims[0, best_iid]))
    

In [ ]:
hit_ratio(recs, test_dict)

In [237]:
model = ContRS_cosine(iids, uids, X_g, X_u)
recs = model.get_batch_recs(N)

hit_ratio(recs, test_dict)

0.0019075568598679383

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
sims = cosine_similarity(X_u[0], X_g)

best_iid = np.argsort(sims[0])[-(10+1):-1]
true_x_iid = [iids.ix[x_iid,'iid'] for x_iid in best_iid]

recs_u0 = dict(zip(true_x_iid, sims[0, best_iid]))


In [ ]:
true_iid = iids.ix[6142,'iid']

item_profile = tfidf.transform(training_df.loc[training_df.iid==true_iid].normalized)

ftr_id_to_term = {ftr_id: term for term, ftr_id in tfidf.vocabulary_.items()}

for ftr_id, score in sorted(zip(item_profile.indices, item_profile.data), key=lambda x: x[1], reverse=True):
    print(ftr_id_to_term[ftr_id], ":", score)

In [ ]:
true_uid = uids[0]

u_mean_rating = training_df.loc[training_df.uid == true_uid].rating.mean()
u_iids = training_df.loc[training_df.uid == true_uid].iid.unique()

u_profile = []

for iid in u_iids:
    rating = training_df.loc[(training_df.uid == true_uid) & (training_df.iid == iid)].rating.values[0] - u_mean_rating

    tmp = tfidf.transform(training_df.loc[training_df.iid==iid].normalized) * rating
    tmp = csr_matrix(tmp.sum(axis=0))
    u_profile.append(tmp)

u_profile = vstack(u_profile, 'csr')

ftr_id_to_term = {ftr_id: term for term, ftr_id in tfidf.vocabulary_.items()}

for ftr_id, score in sorted(zip(u_profile.indices, u_profile.data), key=lambda x: x[1], reverse=True):
    print(ftr_id_to_term[ftr_id], ":", score)

## 5. Item-based collaborative filtering RS